In [ ]:
import os
import shutil
import glob
from tqdm import tqdm
import numpy as np

In [ ]:
# download data set
!wget http://mrl.cs.vsb.cz/data/eyedataset/mrlEyes_2018_01.zip

--2022-10-25 20:15:54--  http://mrl.cs.vsb.cz/data/eyedataset/mrlEyes_2018_01.zip
Resolving mrl.cs.vsb.cz (mrl.cs.vsb.cz)... 158.196.141.22
Connecting to mrl.cs.vsb.cz (mrl.cs.vsb.cz)|158.196.141.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 341866898 (326M) [application/zip]
Saving to: ‘mrlEyes_2018_01.zip’

mrlEyes_2018_01.zip 100%[===================>] 326.03M  6.84MB/s    in 48s     

2022-10-25 20:16:45 (6.82 MB/s) - ‘mrlEyes_2018_01.zip’ saved [341866898/341866898]



In [ ]:
# extract Data
!unzip -q /content/mrlEyes_2018_01.zip

In [ ]:
# sample image name :s0037_10217_1_1_1_0_0_01
# subject id, image id, gender, glasses, eye state, reflections, lightning, sensor ID
if not os.path.exists('content/eye_data_set'):
    os.makedirs('/content/eye_data_set/close_eyes')
    os.makedirs('/content/eye_data_set/open_eyes')
source_directory = '/content/mrlEyes_2018_01/'
# split data set and copy images to open & closed
for root_dir, sub_dir, files in os.walk(source_directory):
  images = [file for file in files if file.endswith('.png')]
  for im in tqdm(images):
    name_split = im.split('_')
    if name_split[4] == '0' and name_split[6] == '1' and name_split[5] == '0' and name_split[7] != '2' and name_split[3] == '0':
      shutil.copy(src = root_dir + '/' + im, dst = 'eye_data_set/close_eyes')
    elif im.split('_')[4] == '1' and name_split[6] == '1' and name_split[5] == '0' and name_split[7] != '2' and name_split[3] == '0':
      shutil.copy(src = root_dir + '/' + im, dst = 'eye_data_set/open_eyes')

0it [00:00, ?it/s]
100%|██████████| 8884/8884 [00:00<00:00, 40031.50it/s]


In [ ]:
image_classes = ['close_eyes', 'open_eyes']
test_ratio = 0.1
root_path = 'eye_data_set/'

# create split dataset dirs
if not os.path.exists('final_data_set'):
  for im_class in image_classes:
    if not os.path.exists('final_data_set/' + im_class):
      os.makedirs('final_data_set/train/' + im_class)
      os.makedirs('final_data_set/test/' + im_class)

    im_class_path = root_path + im_class
    all_images = os.listdir(im_class_path)
    
    # shuffle all file names
    np.random.shuffle(all_images)

    # split size 90%, 10%
    train_im_names, test_im_names = np.split(np.array(all_images), [int(len(all_images) * (1-test_ratio))])
    train_im_names = train_im_names.tolist()
    test_im_names = test_im_names.tolist()

    print('Image Class :',im_class, '\n Total images : ', len(all_images))
    print('Training set Size : ', len(train_im_names))
    print('Testing set size:',len(test_im_names))
    # copy images to new train/test set folders
    for name in tqdm(train_im_names):
      shutil.copy(im_class_path + '/' + name, 'final_data_set/train/' + im_class)
    for name in tqdm(test_im_names):
      shutil.copy(im_class_path + '/' + name, 'final_data_set/test/' + im_class)
print('All Done.')

    

Image Class : close_eyes 
 Total images :  9931
Training set Size :  8937
Testing set size: 994


100%|██████████| 994/994 [00:00<00:00, 10259.03it/s]


Image Class : open_eyes 
 Total images :  10551
Training set Size :  9495
Testing set size: 1056


100%|██████████| 1056/1056 [00:00<00:00, 10585.83it/s]

All Done.


In [ ]:
# !rm -r final_data_set/

## **Training Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
batchsize=32
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2, zoom_range=0.2,width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory('final_data_set/train',target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory('final_data_set/train',target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 14746 images belonging to 2 classes.
Found 3686 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory('final_data_set/test', target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 2050 images belonging to 2 classes.


In [ ]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 39, 39, 32)   0           ['batch_normalization[0][0]']

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [ ]:
if not os.path.exists('models'):
  os.makedirs('models')
checkpoint = ModelCheckpoint('models',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/5
1841/1843 [============================>.] - ETA: 0s - loss: 0.1981 - accuracy: 0.9254
Epoch 1: val_loss improved from inf to 0.19428, saving model to models


1843/1843 [==============================] - 96s 45ms/step - loss: 0.1979 - accuracy: 0.9255 - val_loss: 0.1943 - val_accuracy: 0.9239 - lr: 0.0010
Epoch 2/5
1843/1843 [==============================] - ETA: 0s - loss: 0.1644 - accuracy: 0.9381
Epoch 2: val_loss improved from 0.19428 to 0.16220, saving model to models


1843/1843 [==============================] - 81s 44ms/step - loss: 0.1644 - accuracy: 0.9381 - val_loss: 0.1622 - val_accuracy: 0.9334 - lr: 0.0010
Epoch 3/5
1841/1843 [============================>.] - ETA: 0s - loss: 0.1545 - accuracy: 0.9437
Epoch 3: val_loss did not improve from 0.16220
1843/1843 [==============================] - 56s 30ms/step - loss: 0.1545 - accuracy: 0.9437 - val_loss: 0.1665 - val_accuracy: 0.9280 - lr: 0.0010
Epoch 4/5
1842/1843 [============================>.] - ETA: 0s - loss: 0.1404 - accuracy: 0.9473
Epoch 4: val_loss improved from 0.16220 to 0.16194, saving model to models


1843/1843 [==============================] - 85s 46ms/step - loss: 0.1403 - accuracy: 0.9473 - val_loss: 0.1619 - val_accuracy: 0.9318 - lr: 0.0010
Epoch 5/5
1843/1843 [==============================] - ETA: 0s - loss: 0.1451 - accuracy: 0.9479
Epoch 5: val_loss improved from 0.16194 to 0.15152, saving model to models


1843/1843 [==============================] - 81s 44ms/step - loss: 0.1451 - accuracy: 0.9479 - val_loss: 0.1515 - val_accuracy: 0.9429 - lr: 0.0010


## Model Evaluation

In [ ]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


0.12420584261417389
0.9524616599082947


In [ ]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


0.15949852764606476
0.9395008087158203


In [ ]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

model.save('eye_blink.h5')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


0.08574564009904861
0.9673170447349548


In [ ]:
!zip -r /content/models /content/models/

  adding: content/models/ (stored 0%)
  adding: content/models/keras_metadata.pb (deflated 96%)
  adding: content/models/assets/ (stored 0%)
  adding: content/models/saved_model.pb (deflated 92%)
  adding: content/models/variables/ (stored 0%)
  adding: content/models/variables/variables.index (deflated 79%)
  adding: content/models/variables/variables.data-00000-of-00001 (deflated 8%)


In [ ]:
from google.colab import files
model.save('eye_blink.h5')

In [ ]:

files.download('eye_blink_50_epoch.h5')

FileNotFoundError: ignored